In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from functools import reduce
import pandas as pd
import pickle
from typing import List
import spacy
import string
from unidecode import unidecode

In [2]:
spark = SparkSession.builder.appName('Exploracao').getOrCreate()

24/03/03 00:16:05 WARN Utils: Your hostname, rodrigo-nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp8s0)
24/03/03 00:16:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 00:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Comentários

In [3]:
lista_assunto = [
    'power_bi',
    'python_and_dados',
    'cities_skylines',
    'cities_skylines_2',
    'linux',
    'linux_gamming',
    'genshin_impact',
    'zelda'
]

datas = [
    'extracao_data_2024_01_17',
    'extracao_data_2024_01_18',
    'extracao_data_2024_01_19',
    'extracao_data_2024_01_20',
    'extracao_data_2024_01_21',
    'extracao_data_2024_01_22',
    'extracao_data_2024_01_23',
]

In [4]:
df_comentarios = spark.read.json(f'../dados/bronze/assunto_power_bi/extracao_data_2024_01_17/comentarios/req_comentarios.json')

In [5]:
df_comentarios.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- canReply: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- isPublic: boolean (nullable = true)
 |    |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |    |-- etag: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- kind: string (nullable = true)
 |    |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |    |-- authorChannelUrl: str

In [6]:
def abrir_comentarios( lista_datas: List, lista_assunto: List) -> List[DataFrame]:
    lista_df = []
    for assunto in lista_assunto:
        for datas in lista_datas:
            df_comentarios = spark.read.json(f'../dados/bronze/assunto_{assunto}/{datas}/comentarios/req_comentarios.json')
            
            dataframe = df_comentarios.select(
                    f.explode('items').alias('item'),
                    'data_extracao',
                    'etag',
                    'kind',
                    'pageInfo',
                    
                ).select(
                    'item.id',
                    'item.snippet.channelId',
                    'item.snippet.videoId',
                    'item.snippet.topLevelComment.snippet.textOriginal',
                    
                
            )
            lista_df.append(dataframe)
    return lista_df


In [7]:
lista_df = abrir_comentarios(lista_datas=datas, lista_assunto=lista_assunto)

In [8]:
df_final = reduce(lambda df1, df2: df1.union(df2), lista_df)
df_final.show(truncate=False)

24/03/03 00:16:40 WARN DAGScheduler: Broadcasting large task binary with size 1955.9 KiB
+--------------------------+------------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                        |channelId               |videoId    |textOriginal                                                                                            

In [9]:
df_comentarios_pandas = df_final.toPandas()
df_comentarios_pandas.head()

24/03/03 00:16:44 WARN DAGScheduler: Broadcasting large task binary with size 1943.6 KiB


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise\nLIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...


In [10]:
df_comentarios_pandas[['id', 'channelId', 'videoId', 'textOriginal']] = df_comentarios_pandas[['id', 'channelId', 'videoId', 'textOriginal']].astype('string') 

In [11]:
df_comentarios_pandas = df_comentarios_pandas.drop_duplicates()

# Resposta Comentários

In [12]:
def abrir_respostas_comentarios( lista_datas: List, lista_assunto: List) -> List[DataFrame]:
    lista_df = []
    for assunto in lista_assunto:
        for datas in lista_datas:
            df_resposta_comentarios = spark.read.json(f'../dados/bronze/assunto_{assunto}/{datas}/resposta_comentarios/resposta_comentarios.json')
            
            dataframe = df_resposta_comentarios.select(
                f.explode('items').alias('item'),
                            
            ).select(
                'item.id',
                'item.snippet.channelId',
                'item.snippet.textOriginal',
                'item.snippet.parentId'
            )
            lista_df.append(dataframe)
    return lista_df

In [13]:
lista_df_resposta_comentarios = abrir_respostas_comentarios(lista_assunto=lista_assunto, lista_datas=datas)

In [14]:
df_resposta_comentarios_final = reduce(lambda df1, df2: df1.union(df2), lista_df_resposta_comentarios)
df_resposta_comentarios_final.show(truncate=False)

24/03/03 00:17:04 WARN DAGScheduler: Broadcasting large task binary with size 1210.0 KiB
+-------------------------------------------------+------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|id                                               |channelId               |textOriginal                                                                                                                                                                                                                                                                                                                                             |parentId                  |
+----------

In [15]:
df_resposta_comentarios_final_pandas = df_resposta_comentarios_final.toPandas()

24/03/03 00:17:06 WARN DAGScheduler: Broadcasting large task binary with size 1197.6 KiB


In [16]:
spark.stop()

In [17]:
df_resposta_comentarios_final_pandas[['id', 'channelId', 'textOriginal', 'parentId']] = df_resposta_comentarios_final_pandas[['id', 'channelId', 'textOriginal', 'parentId']].astype('string')

In [18]:
df_resposta_comentarios_final_pandas = df_resposta_comentarios_final_pandas.drop_duplicates()

# Concatenando dataframes de comentários

In [19]:
df_comentarios_pandas.head()

,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...


In [20]:
df_resposta_comentarios_final_pandas.head()


,id,channelId,textOriginal,parentId
0,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgPKKBrNVO,UCcuLEJUZy4jo9iFvngkoBUg,Há pelo menos dois,UgzZff_AwK5aYnauP6h4AaABAg
1,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgQElZCdaG,UCcuLEJUZy4jo9iFvngkoBUg,grato pelo retorno​@@SaladeEmprego,UgzZff_AwK5aYnauP6h4AaABAg
2,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgQFicpN39,UCcuLEJUZy4jo9iFvngkoBUg,grato pelo retorno​@@SaladeEmprego,UgzZff_AwK5aYnauP6h4AaABAg
3,UgxIk3_tEF6SSnEjGDV4AaABAg.9zgB2WyYMq69zgPIGQSki6,UCcuLEJUZy4jo9iFvngkoBUg,Sucesso!,UgxIk3_tEF6SSnEjGDV4AaABAg
4,UgzQwjfcCgBDQVEgCkt4AaABAg.9zfoXBRb8Lb9zgBDjFoOcm,UCcuLEJUZy4jo9iFvngkoBUg,Normalmente a entrevista em grupo costuma ser ...,UgzQwjfcCgBDQVEgCkt4AaABAg


In [21]:
df_comentarios_completos = pd.merge(
    left=df_comentarios_pandas, 
    right=df_resposta_comentarios_final_pandas,
    how='left',
    left_on='id',
    right_on='parentId',

)

In [22]:
df_comentarios_completos.head()

,id_x,channelId_x,videoId,textOriginal_x,id_y,channelId_y,textOriginal_y,parentId
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE",UgzOiQoDWs3JCBPLywx4AaABAg.9zgaq6XZ-0X9ziR1uLspMi,UCnTTl5-22qOKRSVmuGwm1Lw,Valeu 😀,UgzOiQoDWs3JCBPLywx4AaABAg
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra,<NA>,<NA>,<NA>,<NA>
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11,Ugze4k37m59w4PSiHzp4AaABAg.9zfem8NWX6O9zfhNt0aRfp,UCnTTl5-22qOKRSVmuGwm1Lw,"boa, mais um infra pra gente falar aqui no can...",Ugze4k37m59w4PSiHzp4AaABAg
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...,UgyDJGKKqh8emjbSF054AaABAg.9zfasc_aByl9zfbBVmB5Xx,UCnTTl5-22qOKRSVmuGwm1Lw,isso faz com que o Patrimônio do fundo fique p...,UgyDJGKKqh8emjbSF054AaABAg


In [23]:
df_comentarios_completos.shape

(14348, 8)

In [24]:
df_comentarios_completos_um = df_comentarios_completos[['id_x', 'channelId_x', 'videoId', 'textOriginal_x']]
df_comentarios_completos_um.rename(columns={
    'id_x': 'id',
    'channelId_x': 'channelId',
    'textOriginal_x': 'textOriginal'
}, inplace=True)
df_comentarios_completos_um

/tmp/ipykernel_78761/1939900459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comentarios_completos_um.rename(columns={


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...
...,...,...,...,...
14343,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...
14344,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...
14345,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...
14346,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...


In [25]:
df_comentarios_completos_dois = df_comentarios_completos[['id_x', 'channelId_x', 'videoId', 'textOriginal_y']]
df_comentarios_completos_dois.rename(columns={
    'id_x': 'id',
    'channelId_x': 'channelId',
    'textOriginal_y': 'textOriginal'
}, inplace=True)
df_comentarios_completos_dois

/tmp/ipykernel_78761/112964936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comentarios_completos_dois.rename(columns={


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast..."
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Valeu 😀
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,<NA>
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"boa, mais um infra pra gente falar aqui no can..."
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,isso faz com que o Patrimônio do fundo fique p...
...,...,...,...,...
14343,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,😅
14344,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,eu achava que era edição do Bezzi pra não assu...
14345,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,O Hélder te pega demais?
14346,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,"Ah. ...pode crer ... hahahaha, deve ser um sis..."


In [26]:
df_comentarios_completo = pd.concat([df_comentarios_completos_um, df_comentarios_completos_dois])
df_comentarios_completo

,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...
...,...,...,...,...
14343,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,😅
14344,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,eu achava que era edição do Bezzi pra não assu...
14345,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,O Hélder te pega demais?
14346,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,"Ah. ...pode crer ... hahahaha, deve ser um sis..."


# Abrindo dataframe de canal, vídeo

In [27]:
df_assunto = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_gerais.parquet', columns=['ASSUNTO', 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO'])
df_assunto

,ASSUNTO,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO
0,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts
1,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024
2,assunto_python_and_dados,UCiYrw6iAqlYXvP7no7h4tUQ,Rodolfo De Nadai,s9hUoajuWwc,Python: Structure Data Types
3,assunto_python_and_dados,UCNblNh-u0nu5mtXT84Wgdiw,Mestres da Informática,j1-IbFM7TtE,🔴O Novo Ciclo - 1° Fase | O Detetive Virtual
4,assunto_python_and_dados,UCU5vSFBBO_cp6ie0OG74b2g,Refatorando,5Rl2CnW5ZFc,Como é o dia a dia de trabalho de um Programador
...,...,...,...,...,...
12063,assunto_cities_skylines_2,UCGved3ynk6ZhUvSKxzgweTw,Jogando,tjZZTkpmPo4,"07 Itaquera - Cities Skylines 2 UNIVERSIDADE, ..."
12064,assunto_cities_skylines_2,UCdrd9d0-1p8sgXOvJDc-naA,FLAVIOSUICO GAMES,J0O_HqOo4UE,Cities Skyline 2 - INTERSECÇÃO ENORME - Gamepl...
12065,assunto_cities_skylines_2,UCDqzBrUOm7guqqyU8eDackg,BUG CITIES,AHKQdCqYoHI,Cidade (Mods) CITIES SKYLINES I (iniciante)| C...
12066,assunto_cities_skylines_2,UCDqzBrUOm7guqqyU8eDackg,BUG CITIES,AHKQdCqYoHI,Cidade (Mods) CITIES SKYLINES I (iniciante)| C...


In [28]:
df_assunto = df_assunto.drop_duplicates()

In [29]:
df_comentarios_completo_assunto = pd.merge(
    left=df_assunto,
    right=df_comentarios_completo,
    how='inner',
    left_on='ID_VIDEO',
    right_on='videoId'
)
df_comentarios_completo_assunto

,ASSUNTO,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO,id,channelId,videoId,textOriginal
0,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024,UgwVZnxz6RRW7H69Q6R4AaABAg,UC70kawNmZCDeZDZBbeiolHA,S8FcfOPjFJY,Toooop
1,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024,UgwoxPxWmFNyQ1phrUZ4AaABAg,UC70kawNmZCDeZDZBbeiolHA,S8FcfOPjFJY,Conteúdo de qualidade e bem explicado.
2,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024,Ugz7sxak3OoggtRI4iN4AaABAg,UC70kawNmZCDeZDZBbeiolHA,S8FcfOPjFJY,O brabo 👏🏼
3,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024,Ugz2jSdhr1LqxI5BNQ54AaABAg,UC70kawNmZCDeZDZBbeiolHA,S8FcfOPjFJY,🚀👏🏻👏🏻👏🏻
4,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024,UgyVyL5Ws1ZYtJyL5nN4AaABAg,UC70kawNmZCDeZDZBbeiolHA,S8FcfOPjFJY,👏👏👏
...,...,...,...,...,...,...,...,...,...
34619,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugz0tT97md19_R-yfgV4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,Surra mtooooooo Hahahah e isso é de 21 anos at...
34620,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugz0tT97md19_R-yfgV4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,"Tudo menos o cas, o cas de the Sims 4 é incrível."
34621,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugz0tT97md19_R-yfgV4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,so nao é como o do ts2@@chipsdiversos9427
34622,assunto_cities_skylines_2,UCY9SX2eszZG1C63NpRARA9g,Nicolas Gameplays,Ctq8ZpvDNb0,"TOP GAME DISPONIVEL AGORA, PLAGUE TALE REQUIE...",Ugy580FmSV6xkZJ-tM94AaABAg,UCY9SX2eszZG1C63NpRARA9g,Ctq8ZpvDNb0,Jogão meu brother as duas série o 1 e o 2 zere...


In [30]:
nlp = spacy.load("pt_core_news_sm")

In [31]:
doc = nlp('Tudo Fazer')

In [32]:
num_tokens = len(doc)
num_tokens

2

In [33]:
from collections import Counter
term_freq = Counter(token.text for token in doc if not token.is_stop and not token.is_punct)
term_freq

Counter()

In [34]:
nlp.vocab.length

352

In [35]:
named_entities = [(entity.text, entity.label_) for entity in doc.ents]
named_entities

[('Fazer', 'LOC')]

In [36]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [37]:
pos_tags = [(token.text, token.pos_) for token in doc]
pos_tags

[('Tudo', 'PRON'), ('Fazer', 'PROPN')]

In [38]:
dependency_relations = [(token.text, token.dep_, token.head.text) for token in doc]
dependency_relations

[('Tudo', 'ROOT', 'Tudo'), ('Fazer', 'flat:name', 'Tudo')]

In [39]:
for token in doc:
    print(token.text, token.lemma_)

Tudo tudo
Fazer Fazer


In [40]:
for token in doc:
    print(token.text, token.pos_)

Tudo PRON
Fazer PROPN


In [41]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Fazer LOC


In [42]:
def remove_stop_words(sentence):
    try:
        nlp = spacy.load("pt_core_news_sm")
        sentence = unidecode(sentence).strip().lower()
        for c in string.punctuation:
            sentence = sentence.replace(c, '')
        doc = nlp(sentence)
        filtered_tokens = [token for token in doc if not token.is_stop]
        palavras = list(filtered_tokens)
        lista_palavras = [palava for palava in palavras if len(palava) > 1]
        return lista_palavras
    except:
        return ''

In [43]:
texto = 'O brab avião 👏🏼'
remove_stop_words(texto)

[brab, aviao]

In [44]:
lista_assunto = [
    'assunto_power_bi',
    'assunto_python_and_dados',
    'assunto_cities_skylines',
    'assunto_cities_skylines_2',
    'assunto_linux',
    'assunto_linux_gamming',
    'assunto_genshin_impact',
    'assunto_zelda'
]

In [45]:
n = 10
df_comentarios_completo_assunto['ID'] = df_comentarios_completo_assunto.index+1
df_comentarios_completo_assunto['PARTICOES'] = df_comentarios_completo_assunto['ID'] // n



In [46]:
df_comentarios_completo_assunto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34624 entries, 0 to 34623
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ASSUNTO       34624 non-null  string
 1   ID_CANAL      34624 non-null  string
 2   NM_CANAL      34624 non-null  string
 3   ID_VIDEO      34624 non-null  string
 4   TITULO_VIDEO  34624 non-null  string
 5   id            34624 non-null  string
 6   channelId     34624 non-null  string
 7   videoId       34624 non-null  string
 8   textOriginal  25674 non-null  string
 9   ID            34624 non-null  int64 
 10  PARTICOES     34624 non-null  int64 
dtypes: int64(2), string(9)
memory usage: 2.9 MB


In [47]:
lista_particoes = df_comentarios_completo_assunto['PARTICOES'].unique().tolist()

In [48]:
df_comentarios_completo_assunto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34624 entries, 0 to 34623
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ASSUNTO       34624 non-null  string
 1   ID_CANAL      34624 non-null  string
 2   NM_CANAL      34624 non-null  string
 3   ID_VIDEO      34624 non-null  string
 4   TITULO_VIDEO  34624 non-null  string
 5   id            34624 non-null  string
 6   channelId     34624 non-null  string
 7   videoId       34624 non-null  string
 8   textOriginal  25674 non-null  string
 9   ID            34624 non-null  int64 
 10  PARTICOES     34624 non-null  int64 
dtypes: int64(2), string(9)
memory usage: 2.9 MB


In [49]:
df_comentarios_completo_assunto.to_parquet('../dados/ouro/comentarios.parquet')